Ideal abstraction: 
--The user specifies some "frequency" (in seconds, examples, batches, or epochs) at which certain metrics are calculated (can use different units, freqs). 
--Metrics can be built-in or custom (user provides a func for calculating them, given a model and DataLoader) 
--Everything else (printing, logging to TB, checkpointing, etc.) operates over this metrics dict
--For checkpointing, the user can specify which metric to use and whether to min or max it.

full name of metrics is split/metric (e.g., valid/accuracy)
split is assumed based on which logger is being used, assumed to be valid for checkpointer
    but you can explicitly indicate otherwise if you'd like!

all metrics (standard and custom) are dumped to metrics_dict
all metrics values are pulled from metrics_dict

Train loss gets reported continuously with tqdm progress bar?

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from synthetic.generate import singletask_synthetic

n = 10000
m = 10
k = 2
D, L, X, Y, _ = singletask_synthetic(n, m, k)
    
from metal.utils import split_data
Xs, Ys, Ls, Ds = split_data(X, Y, L, D, splits=[0.8, 0.1, 0.1], stratify_by=Y, seed=123)    

In [7]:
from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)

Y_train_ps = mv.predict_proba(Ls[0])
# scores = mv.score((Ls[1], Ys[1]), metric=['precision', 'recall', 'f1'])

In [19]:
from metal.end_model import EndModel

end_model = EndModel([1000,10,2])
end_model.train_model(
    (Xs[0], Y_train_ps), 
    valid_data=(Xs[1], Ys[1]), 
    l2=0.01, 
    batch_size=256, 
    n_epochs=10, 
    tensorboard=False,
    checkpoint=False,
    log_unit='epochs',
    log_train_every=1,
    log_valid_every=3,
)


Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=1000, out_features=10, bias=True)
    (1): ReLU()
  )
  (2): Linear(in_features=10, out_features=2, bias=True)
)

[1 epochs]: train/loss=16.502
[2 epochs]: train/loss=9.994
[3 epochs]: train/loss=6.058, valid/accuracy=0.996
[4 epochs]: train/loss=9.207
[5 epochs]: train/loss=9.426
[6 epochs]: train/loss=7.607, valid/accuracy=0.994
[7 epochs]: train/loss=2.193
[8 epochs]: train/loss=3.012
[9 epochs]: train/loss=2.129, valid/accuracy=0.988
Finished Training
Accuracy: 0.991
        y=1    y=2   
 l=1    493     6    
 l=2     3     498   
